In [4]:
import os
from selenium import webdriver
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup, Comment
import requests
import time
import urllib
import urllib.request
import pandas as pd

In [5]:
season = 2023
month = "december"

In [6]:
url = f"https://www.basketball-reference.com/leagues/NBA_{season}_games.html"

path = '/E:/Projects/NBA Prediction/chromedriver.exe'

options = webdriver.ChromeOptions()

options.add_argument('--ignore-certificate-errors')
options.add_argument('--ignore-ssl-errors')
driver = webdriver.Chrome(options=options, executable_path=path)
driver.minimize_window()
driver.get(url)

all_months = driver.find_elements(By.XPATH, "//div[@class='filter']//a")
all_month_links = []
all_month_filename = []
for months in all_months:
    time.sleep(3)
    all_month_links.append(months.get_attribute('href'))
    all_month_filename.append(months.text.lower())

driver.quit()

C:\Users\ryuos\AppData\Local\Temp\ipykernel_45312\4057414326.py:9: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(options=options, executable_path=path)


In [7]:
html = ""
for i in range(len(all_month_links)):
    if month in all_month_links[i]:
        html = all_month_links[i]

In [8]:
def scrape(month):
    time.sleep(15)

    url = month

    result = requests.get(url)
    doc = BeautifulSoup(result.text, "html.parser")

    links = doc.find_all("a")

    hrefs = [l.get("href") for l in links]
    box_scores = [l for l in hrefs if l and "boxscore" in l and ".html" in l]
    name_of_games = box_scores[:]
    for i in range(len(name_of_games)):
        name_of_games[i] = name_of_games[i][11:23]

    box_scores = [f"https://www.basketball-reference.com{l}" for l in box_scores]
    

    return box_scores, name_of_games

In [10]:
box_scores, name_of_games = scrape(html)

In [11]:
games_to_scrape = []
names_to_scrape = []

for i in range(len(name_of_games)):
    if os.path.exists("./html/"+ name_of_games[i] + ".html"):
        pass
    else:
        games_to_scrape.append(box_scores[i])
        names_to_scrape.append(name_of_games[i])

In [12]:
def run_monthly(all_boxes, name):
    for i in range(len(all_boxes)):
        time.sleep(3)
        opener = urllib.request.FancyURLopener({})
        file = "html/" + name[i] + ".html"
        url = all_boxes[i]
        opener.retrieve(url, file)

In [13]:
run_monthly(games_to_scrape, names_to_scrape)

C:\Users\ryuos\AppData\Local\Temp\ipykernel_45312\1398164903.py:4: DeprecationWarning: FancyURLopener style of invoking requests is deprecated. Use newer urlopen functions/methods
  opener = urllib.request.FancyURLopener({})


In [14]:
import os
import pandas as pd
from bs4 import BeautifulSoup, Comment
from tqdm import tqdm

In [15]:
SCORE_DIR = "E:/Projects/NBA Prediction/html"
new_box_scores = []
for i in range(len(names_to_scrape)):
    names_to_scrape[i] = names_to_scrape[i] + ".html"
    new_box_scores.append(os.path.join(SCORE_DIR, names_to_scrape[i]))

In [16]:
def parse_html(box_score):
    with open(box_score, "r", encoding='utf-8') as f:
        html = f.read()
    
    soup = BeautifulSoup(html)
    comments = soup.find_all(string=lambda text: isinstance(text, Comment))

    [s.decompose() for s in soup.select("tr.over_header")]
    [s.decompose() for s in soup.select("tr.thread")]

    return soup, comments


In [17]:
def read_line_score(soup):
    line_score = pd.read_html(str(soup), attrs={"id": "line_score"})[0]
    cols = list(line_score.columns)
    cols[0] = "team"
    cols[-1] = "total"
    line_score.columns = cols
    
    line_score = line_score[["team", "total"]]
    return line_score

In [18]:
def read_stats(soup, team, stat):
    df = pd.read_html(str(soup), attrs={"id": f"box-{team}-game-{stat}"}, index_col=0)[0]
    df = df.apply(pd.to_numeric, errors="coerce")
    return df

In [19]:
def read_season_info(soup):
    nav = soup.select("#bottom_nav_container")[0]
    hrefs = [a["href"] for a in nav.find_all("a")]
    season = os.path.basename(hrefs[1]).split("_")[0]
    return season

In [20]:
base_cols = None
games = []

for box_score in tqdm(new_box_scores):
    soup, comment = parse_html(box_score)
    line_score = read_line_score(comment)
    teams = list(line_score["team"])

    summaries = []
    for team in teams:
        basic = read_stats(soup, team, "basic")
        advanced = read_stats(soup, team, "advanced")

        totals = pd.concat([basic.iloc[-1,:], advanced.iloc[-1,:]])
        totals.index = totals.index.str.lower()

        maxes = pd.concat([basic.iloc[:-1,:].max(), advanced.iloc[:-1,:].max()])
        maxes.index = maxes.index.str.lower() + "_max"

        summary = pd.concat([totals, maxes])

        if base_cols is None:
            base_cols = list(summary.index.drop_duplicates(keep="first"))
            base_cols = [b for b in base_cols if "bpm" not in b]
        
        summary = summary[base_cols]

        summaries.append(summary)

    summary = pd.concat(summaries, axis=1).T

    game = pd.concat([summary, line_score], axis=1)
    game["home"] = [0, 1]
    game_opp = game.iloc[::-1].reset_index()
    game_opp.columns += "_opp"

    full_game = pd.concat([game, game_opp], axis=1)

    full_game["season"] = read_season_info(soup)

    full_game["date"] = os.path.basename(box_score)[:8]
    full_game["date"] = pd.to_datetime(full_game["date"], format="%Y%m%d")

    full_game["won"] = full_game["total"] > full_game["total_opp"]

    games.append(full_game)

100%|██████████| 7/7 [00:07<00:00,  1.12s/it]


In [21]:
games_df = pd.concat(games, ignore_index=True)
cols1 = list(games_df.columns)

In [22]:
temp = pd.read_csv("nba_games.csv", index_col=0)

cols2 = list(temp.columns)
cols2[1] = "mp"
cols2[36] = "mp_max"
cols2[75] = "mp_opp"
cols2[110] = "mp_max_opp"
temp.columns = cols2

In [23]:
test_cat = [temp, games_df]

In [24]:
result = pd.concat([temp.astype({'date': 'str'}), games_df.astype({'date': 'str'})], ignore_index=True)

In [25]:
result.to_csv("nba_games.csv")